# Lab 3: Generative AI with Ollama - SOLUTIONS

**Duration:** 90-120 minutes | **Difficulty:** Intermediate to Advanced

## Learning Objectives

By the end of this lab, you will be able to:
1. Connect to and use Ollama for local LLM inference
2. Generate text using the Llama model
3. Apply prompt engineering techniques for better results
4. Control generation with temperature and other parameters
5. Build multi-turn conversations with chat history
6. Implement Retrieval-Augmented Generation (RAG)
7. Understand fine-tuning concepts with LoRA and QLoRA

## Setup

In [ ]:
import ollama
import json
import numpy as np
from typing import List, Dict

# Check connection to Ollama
try:
    models = ollama.list()
    print("Connected to Ollama!")
    print("\nAvailable models:")
    for model in models.get('models', []):
        print(f"  - {model['name']}")
except Exception as e:
    print(f"Error connecting to Ollama: {e}")
    print("\nMake sure Ollama is running: ollama serve")

---
# Part 1: Basic Text Generation

In [ ]:
# Basic text generation
response = ollama.generate(
    model='llama3.2',
    prompt='What is machine learning in one sentence?'
)
print("Response:")
print(response['response'])

## Exercise 1.1: Generate Your First Response - SOLUTION

In [ ]:
# SOLUTION
my_response = ollama.generate(
    model='llama3.2',
    prompt='What is an API?'
)
answer = my_response['response']
print("Answer:", answer)

---
# Part 2: Prompt Engineering

## Exercise 2.1: Write a Role-Based Prompt - SOLUTION

In [ ]:
# SOLUTION
chef_prompt = """You are a professional chef who specializes in quick, easy meals 
that anyone can make at home with common ingredients.

Suggest a simple dinner recipe that can be made in under 30 minutes.
Include a list of ingredients and brief cooking instructions."""

recipe_response = ollama.generate(model='llama3.2', prompt=chef_prompt)

if recipe_response:
    print(recipe_response['response'])

## Exercise 2.2: Generate JSON Output - SOLUTION

In [ ]:
# SOLUTION
movie_prompt = """Generate information about a famous movie.

Respond with ONLY valid JSON in this exact format:
{"title": "...", "director": "...", "year": YYYY, "rating": X.X}

The rating should be out of 10. Do not include any other text, just the JSON."""

movie_response = ollama.generate(model='llama3.2', prompt=movie_prompt)

try:
    movie_data = json.loads(movie_response['response'].strip())
except:
    movie_data = None

if movie_data:
    print("Movie data:")
    for key, value in movie_data.items():
        print(f"  {key}: {value}")

---
# Part 3: Generation Parameters

## Exercise 3.1: Experiment with Temperature - SOLUTION

In [ ]:
# SOLUTION
creative_prompt = "Suggest a creative and unique name for a coffee shop."

low_temp_response = ollama.generate(
    model='llama3.2',
    prompt=creative_prompt,
    options={'temperature': 0.2}
)

high_temp_response = ollama.generate(
    model='llama3.2',
    prompt=creative_prompt,
    options={'temperature': 0.9}
)

print("Low temp (0.2):")
print(low_temp_response['response'])
print("\nHigh temp (0.9):")
print(high_temp_response['response'])

---
# Part 4: Chat Conversations

## Exercise 4.1: Create a Multi-Turn Conversation - SOLUTION

In [ ]:
# SOLUTION
messages = [
    {'role': 'user', 'content': 'My favorite color is blue.'}
]

response1 = ollama.chat(model='llama3.2', messages=messages)

if response1:
    print("Assistant:", response1['message']['content'])
    
    messages.append(response1['message'])
    messages.append({'role': 'user', 'content': 'What is my favorite color?'})
    
    response2 = ollama.chat(model='llama3.2', messages=messages)
    
    if response2:
        print("\nAssistant:", response2['message']['content'])

## Exercise 4.2: Create a Specialized Chatbot - SOLUTION

In [ ]:
# SOLUTION
system_prompt = """You are a helpful and patient Python programming tutor. 
You explain concepts clearly using simple language and always provide code examples.
When answering questions, you break down complex topics into easy-to-understand steps.
You encourage learning and provide helpful tips."""

messages = [
    {'role': 'system', 'content': system_prompt},
    {'role': 'user', 'content': 'What is a list comprehension in Python?'}
]

tutor_response = ollama.chat(model='llama3.2', messages=messages)

if tutor_response:
    print("Python Tutor:")
    print(tutor_response['message']['content'])

---
# Part 5: Building a Simple Application

## Exercise 5.1: Build a Sentiment Analyzer - SOLUTION

In [ ]:
# SOLUTION
def analyze_sentiment(text):
    prompt = f"""Analyze the sentiment of the following text.

Text: {text}

Respond with ONLY valid JSON in this exact format:
{{"sentiment": "positive/negative/neutral", "confidence": "high/medium/low"}}

Do not include any other text."""
    
    response = ollama.generate(
        model='llama3.2',
        prompt=prompt,
        options={'temperature': 0.1}
    )
    
    try:
        return json.loads(response['response'].strip())
    except:
        return None

test_texts = [
    "I absolutely love this product! Best purchase ever!",
    "This is the worst experience I've ever had.",
    "The weather today is cloudy."
]

for text in test_texts:
    result = analyze_sentiment(text)
    if result:
        print(f"Text: {text[:50]}...")
        print(f"  Sentiment: {result.get('sentiment')}")
        print(f"  Confidence: {result.get('confidence')}")
        print()

## Exercise 5.2: Build a Q&A Bot - SOLUTION

In [ ]:
# SOLUTION
def answer_question(context, question):
    prompt = f"""Answer the question based ONLY on the provided context.
If the answer is not in the context, say "I don't know based on the provided context."

Context:
{context}

Question: {question}

Answer:"""
    
    response = ollama.generate(
        model='llama3.2',
        prompt=prompt,
        options={'temperature': 0.2}
    )
    
    return response['response'].strip()

context = """
Python was created by Guido van Rossum and first released in 1991. 
It emphasizes code readability and uses significant indentation. 
Python supports multiple programming paradigms including procedural, 
object-oriented, and functional programming. The language is named 
after the British comedy group Monty Python.
"""

questions = [
    "Who created Python?",
    "When was Python first released?",
    "What is Python's mascot?"
]

for q in questions:
    answer = answer_question(context, q)
    print(f"Q: {q}")
    print(f"A: {answer}")
    print()

---
# Part 6: Retrieval-Augmented Generation (RAG)

In [ ]:
# Helper functions for RAG
def cosine_similarity(a, b):
    a = np.array(a)
    b = np.array(b)
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_embedding(text):
    response = ollama.embed(model='llama3.2', input=text)
    return response['embeddings'][0]

class SimpleRAG:
    def __init__(self, model='llama3.2'):
        self.model = model
        self.documents = []
        self.embeddings = []
    
    def add_documents(self, docs: List[str]):
        for doc in docs:
            embedding = get_embedding(doc)
            self.documents.append(doc)
            self.embeddings.append(embedding)
        print(f"Added {len(docs)} documents. Total: {len(self.documents)}")
    
    def retrieve(self, query: str, top_k: int = 2) -> List[str]:
        query_embedding = get_embedding(query)
        similarities = []
        for i, doc_embedding in enumerate(self.embeddings):
            sim = cosine_similarity(query_embedding, doc_embedding)
            similarities.append((sim, i))
        similarities.sort(reverse=True)
        top_indices = [idx for _, idx in similarities[:top_k]]
        return [self.documents[i] for i in top_indices]
    
    def query(self, question: str, top_k: int = 2) -> str:
        relevant_docs = self.retrieve(question, top_k)
        context = "\n\n".join(relevant_docs)
        prompt = f"""Use the following context to answer the question. 
If the answer is not in the context, say "I don't have information about that."

Context:
{context}

Question: {question}

Answer:"""
        response = ollama.generate(model=self.model, prompt=prompt, options={'temperature': 0.3})
        return response['response'].strip()

# Create and populate RAG system
rag = SimpleRAG()
knowledge_base = [
    "The Eiffel Tower is located in Paris, France. It was built in 1889 and stands 330 meters tall.",
    "The Great Wall of China is over 21,000 kilometers long and was built over many centuries.",
    "Python programming language was created by Guido van Rossum and released in 1991.",
    "Machine learning is a subset of AI that enables computers to learn from data.",
    "The Amazon rainforest produces about 20% of the world's oxygen."
]
rag.add_documents(knowledge_base)

## Exercise 6.1: Extend the RAG Knowledge Base - SOLUTION

In [ ]:
# SOLUTION
my_documents = [
    "The Olympic Games originated in ancient Greece around 776 BC.",
    "Basketball was invented by James Naismith in 1891 in Springfield, Massachusetts.",
    "The FIFA World Cup is held every four years and is the most watched sporting event.",
    "Tennis uses a scoring system of 15, 30, 40, and game points.",
    "The marathon race is 26.2 miles long, commemorating the legend of Pheidippides."
]

rag.add_documents(my_documents)

my_questions = [
    "Who invented basketball?",
    "How long is a marathon?",
    "When did the Olympic Games start?"
]

for q in my_questions:
    print(f"Q: {q}")
    answer = rag.query(q)
    print(f"A: {answer}")
    print()

## Exercise 6.2: Implement Document Chunking - SOLUTION

In [ ]:
# SOLUTION
def chunk_text(text: str, chunk_size: int = 100, overlap: int = 20) -> List[str]:
    """Split text into overlapping chunks."""
    words = text.split()
    chunks = []
    
    if len(words) <= chunk_size:
        return [text.strip()]
    
    start = 0
    while start < len(words):
        end = min(start + chunk_size, len(words))
        chunk = ' '.join(words[start:end])
        chunks.append(chunk.strip())
        
        if end >= len(words):
            break
        start = end - overlap
    
    return chunks

# Test
long_document = """
Artificial intelligence has transformed the technology landscape dramatically over the past decade. 
Machine learning algorithms now power everything from recommendation systems to autonomous vehicles.
Deep learning, a subset of machine learning, uses neural networks with many layers to learn complex patterns.
Natural language processing enables computers to understand and generate human language.
Computer vision allows machines to interpret and analyze visual information from the world.
Reinforcement learning teaches agents to make decisions through trial and error.
The field continues to advance rapidly, with new breakthroughs announced regularly.
Ethical considerations around AI bias and fairness have become increasingly important.
Researchers are working on making AI systems more transparent and explainable.
The future of AI holds both tremendous promise and significant challenges for society.
"""

chunks = chunk_text(long_document, chunk_size=50, overlap=10)
print(f"Created {len(chunks)} chunks:\n")
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}: {chunk[:80]}...")
    print()

---
# Part 7: Fine-tuning Concepts with LoRA and QLoRA

In [ ]:
# Training data formats demonstration
instruction_format = {
    "instruction": "Summarize the following text in one sentence.",
    "input": "The quick brown fox jumps over the lazy dog. This sentence contains every letter.",
    "output": "A pangram sentence featuring a fox demonstrates all 26 alphabet letters."
}

chat_format = {
    "messages": [
        {"role": "system", "content": "You are a helpful coding assistant."},
        {"role": "user", "content": "How do I reverse a string in Python?"},
        {"role": "assistant", "content": "Use slicing: `reversed_string = original_string[::-1]`"}
    ]
}

print("Instruction Format:")
print(json.dumps(instruction_format, indent=2))

## Exercise 7.1: Create Training Data - SOLUTION

In [ ]:
# SOLUTION - Fitness Coach Training Data
my_training_data = [
    {
        "instruction": "Provide fitness advice for a beginner.",
        "input": "I want to start exercising but I've never worked out before. Where do I begin?",
        "output": "Start with 20-30 minutes of walking 3-4 times per week. Once comfortable, add bodyweight exercises like squats, push-ups, and planks. Focus on consistency over intensity - it's better to do a little regularly than to overdo it and quit."
    },
    {
        "instruction": "Suggest a workout modification.",
        "input": "I have bad knees and can't do regular squats. What can I do instead?",
        "output": "Try wall sits or chair-assisted squats to reduce knee strain. You can also do glute bridges, which work similar muscles without stressing your knees. Swimming and cycling are excellent low-impact alternatives for leg strength."
    },
    {
        "instruction": "Provide nutrition advice for fitness.",
        "input": "What should I eat before a morning workout?",
        "output": "Have a light snack 30-60 minutes before: a banana, toast with peanut butter, or a small bowl of oatmeal. Avoid heavy meals. If exercising within 30 minutes of waking, just have water - your body can use stored energy for moderate workouts."
    },
    {
        "instruction": "Create a simple workout plan.",
        "input": "Can you give me a quick 15-minute home workout?",
        "output": "Try this circuit: 1 minute jumping jacks, 10 squats, 10 push-ups (or knee push-ups), 30-second plank, 10 lunges each leg, rest 30 seconds. Repeat twice. This hits all major muscle groups and gets your heart rate up!"
    },
    {
        "instruction": "Motivate someone who wants to give up.",
        "input": "I've been working out for 2 weeks and haven't lost any weight. Should I quit?",
        "output": "Don't quit! Two weeks is too early to see major results - your body is adapting. Focus on non-scale victories: more energy, better sleep, improved mood. Muscle weighs more than fat, so you might be recomposing. Take progress photos and measurements instead."
    }
]

def validate_training_data(data):
    required_keys = ['instruction', 'input', 'output']
    for i, example in enumerate(data):
        for key in required_keys:
            if key not in example or example[key] is None:
                print(f"Example {i+1} missing '{key}'")
                return False
    print(f"All {len(data)} examples are valid!")
    return True

validate_training_data(my_training_data)

## Exercise 7.2: Design a LoRA Configuration - SOLUTION

In [ ]:
# SOLUTION - LoRA config for fitness coach
my_lora_config = {
    "r": 16,                # Medium rank - good balance for instruction following
    "lora_alpha": 32,       # 2x the rank as recommended
    "target_modules": ["q_proj", "k_proj", "v_proj", "o_proj"],
    "lora_dropout": 0.05,   # Light dropout for regularization
}

my_training_config = {
    "num_epochs": 3,        # Enough for small dataset to converge
    "batch_size": 4,        # Small batch for limited GPU memory
    "learning_rate": 2e-4,  # Standard LoRA learning rate
    "warmup_ratio": 0.05,   # 5% warmup
}

print("Fitness Coach LoRA Config:")
print(json.dumps(my_lora_config, indent=2))
print("\nTraining Config:")
print(json.dumps(my_training_config, indent=2))

---
# Lab Complete!

## Summary

You learned:
- **Basic Generation**: Use `ollama.generate()` for text completion
- **Prompt Engineering**: Role-based prompts, structured output, JSON responses
- **Parameters**: Control creativity with temperature
- **Chat API**: Multi-turn conversations with `ollama.chat()`
- **Applications**: Build summarizers, sentiment analyzers, and Q&A bots
- **RAG**: Implement retrieval-augmented generation with embeddings
- **Fine-tuning**: Understand LoRA/QLoRA for efficient model adaptation